In [27]:
import os
import spotipy
import spotipy.util as util
import time
import csv
import pandas as pd
import sys
import pickle

In [49]:
scope = 'user-library-read'
username = 'your_spotify_username'
token = util.prompt_for_user_token(username,scope,client_id=os.environ['SPOTIPY_CLIENT_ID'],client_secret=os.environ['SPOTIPY_CLIENT_SECRET'],redirect_uri=os.environ['SPOTIPY_REDIRECT_URI'])
print(token)
sp = spotipy.Spotify(auth=token)

BQB6jSON-IuPj-vAL19v60BpN7HbjTFCgJ40XnaM-_w961binSwnrEjAtHsANd1dxEW7r7ZZvcBEgi4VMaQDgVCo5-JgsTaqXUPYug0R-jNESXHsGCebw_XszRSHiHQZi2aKHarvNHP559E04O9k1Cy3wXCJlCE


In [13]:
track_ids = []

In [14]:
with open('song_ids_DO_NOT_MODIFY.csv') as csvfile:
    song_reader = csv.reader(csvfile, delimiter=',')
    for row in song_reader:
        track_ids.append(row[0])

In [15]:
print(track_ids[0:10])

['3HzSqraFqZ8aeoL97E52c2', '3kS6UnhXhy1Wa5eG8oj51G', '2VRArF8Mnmi1dHiHr1DlNE', '7LQbvl0Xie5RE3ZnPtQtK7', '6zXPfmbRwPKhDMQtZID9ed', '19XDJlw6WxC4XOKC1LGKid', '6UBQONiLH8kNmmB8UrV2Ef', '0uQp5uVoeiERuDd9ld0eqw', '3Xls4cNOwy01dtrNXb1inG', '5gA9Xn8oPts2aewPgxVkPD']


In [52]:
features = sp.audio_features('4kV4N9D1iKVxx1KLvtTpjS')
analysis = sp.audio_analysis('4kV4N9D1iKVxx1KLvtTpjS')
print(analysis['meta'])

{'analyzer_version': '4.0.0', 'platform': 'Linux', 'detailed_status': 'OK', 'status_code': 0, 'timestamp': 1550289667, 'analysis_time': 5.96835, 'input_process': 'libvorbisfile L+R 44100->22050'}


In [17]:
def get_audio_features_and_analysis_from_song_id(sp, song_id):
    features = sp.audio_features(song_id)
    analysis = sp.audio_analysis(song_id)
    return (features, analysis)

In [18]:
def remove_string_keys_from_analysis(analysis):
    '''
    analysis: the dictionary returned from sp.audio_analysis(song_id)
    Remove the strings from the analysis because they take up a lot of space
    return: analysis but with said keys removed
    '''
    for key in ['codestring', 'echoprintstring', 'synchstring', 'rhythmstring']:
        analysis['track'].pop(key, None)
    return analysis

In [29]:
def save_song_features(songs_dictionary, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(songs_dictionary, f)

In [35]:
def get_audio_features_and_analysis_from_songs(sp, track_ids, start_index, end_index, block_size, consecutive_errors = 0, file_number = 1):
    '''
    sp: Spotipy Object
    track_ids: List of song ids
    start_index, end_index: only consider track_ids[start_index : end_index]
    return: a list of the form [ (track_id, features, analysis), ... ]
    '''
    if consecutive_errors > 25:
        print("Too many consecutive errors.")
        return
    try:
        song_features_and_analysis = {}
        for idx, track_id in enumerate(track_ids[start_index:end_index]):
            features, analysis = get_audio_features_and_analysis_from_song_id(sp, track_id)
            analysis = remove_string_keys_from_analysis(analysis)
            song_features_and_analysis[track_id] = (features, analysis)
            # Every 100 iterations Sleep for 5 seconds to not overload API
            # Also save progress along the way to a pickle
            if (idx+1) % block_size == 0:
                print("SONG ID IS: ", track_id)
                time.sleep(1)
                save_song_features(song_features_and_analysis, 'songs' + str(file_number))
                print("Songs left: ", end_index - (start_index+idx))
                song_features_and_analysis = {}
                start_index += block_size
                file_number += block_size
            consecutive_errors = 0
            
    except Exception as e:
        print(e)
        print("API called failed")
        print("Last song index was: ", start_index)
        print("Calling function again ")
        #Call the fucntion again from where we left off
        return get_audio_features_and_analysis_from_songs(sp, track_ids, start_index, end_index, consecutive_errors + 1, file_number)
    return 'Success'

In [47]:
get_audio_features_and_analysis_from_songs(sp, track_ids, 0, 1000, 100)

SONG ID IS:  6vo4fRnePZ8UwrlFhyX1Gw
Songs left:  1000
SONG ID IS:  2Hc85u1mdj6nhJ1za7O1m6
Songs left:  900
SONG ID IS:  2ya9d4B06EQ8Fpj5SSqiTH
Songs left:  800
SONG ID IS:  3rYgnMG0aib8mT5NjwwCTT
Songs left:  700
SONG ID IS:  2tgr5xHE39CBIRfeU6U4nV
Songs left:  600
SONG ID IS:  5byJBGDhkVXs81MS3b3Ncq
Songs left:  500
SONG ID IS:  3ACQkIMv6nGYHRpPGyS4BK
Songs left:  400
SONG ID IS:  490afAlqNYXTwEonFlhg7V
Songs left:  300
SONG ID IS:  4LDVjxkzOKa7YUxjO1x74S
Songs left:  200
SONG ID IS:  5yqJhxkeQoyGrBjVWhXlqN
Songs left:  100


'Success'

In [45]:
with open('songs3' + '.pkl', 'rb') as f:
    diction =  pickle.load(f)

In [46]:
print(diction.keys())

dict_keys(['2VRArF8Mnmi1dHiHr1DlNE', '7LQbvl0Xie5RE3ZnPtQtK7'])


In [44]:
print(diction['3HzSqraFqZ8aeoL97E52c2'][1])

{'meta': {'analyzer_version': '4.0.0', 'platform': 'Linux', 'detailed_status': 'OK', 'status_code': 0, 'timestamp': 1444672033, 'analysis_time': 4.96444, 'input_process': 'libvorbisfile L+R 44100->22050'}, 'track': {'num_samples': 2938530, 'duration': 133.26667, 'sample_md5': '', 'offset_seconds': 0, 'window_seconds': 0, 'analysis_sample_rate': 22050, 'analysis_channels': 1, 'end_of_fade_in': 0.2619, 'start_of_fade_out': 130.21751, 'loudness': -14.266, 'tempo': 95.759, 'tempo_confidence': 0.769, 'time_signature': 4, 'time_signature_confidence': 1.0, 'key': 2, 'key_confidence': 0.285, 'mode': 1, 'mode_confidence': 0.43, 'code_version': 3.15, 'echoprint_version': 4.12, 'synch_version': 1.0, 'rhythm_version': 1.0}, 'bars': [{'start': 1.30929, 'duration': 2.50712, 'confidence': 0.528}, {'start': 3.81641, 'duration': 2.4727, 'confidence': 0.556}, {'start': 6.28911, 'duration': 2.44795, 'confidence': 0.523}, {'start': 8.73706, 'duration': 2.49212, 'confidence': 0.355}, {'start': 11.22918, 'd